1st attempt: [530, 163, 110, 146, 594, 109, 138, 115, 157, 533, 94, 145, 155, 164, 200, 129, 179, 201, 241, 522]

2nd attempt: [276, 394, 301, 595, 404, 311, 355, 404, 437, 394, 524, 357, 393, 374, 501, 237, 509, 356]

In [10]:
import numpy as np
from skimage import io
from SegmentationStatisticsCollector import convert_cell_labels_to_meshes, process_labels
import trimesh as tm
from skimage import io
from napari_process_points_and_surfaces import label_to_surface
import napari
import os
import tqdm

### Preprocessing of labels:

In [3]:
path_to_dir = "/nas/groups/iber/Users/Federico_Carrara/create_meshes/data/curated_labels_clean/"
image_name = "lung_new_sample_b_curated_segmentation_central_crop_relabeled_filled.tif"
img_path = os.path.join(path_to_dir, image_name)

save_dir = "../3d_tissues_preprocessing_and_segmentation/meshes/lung_new_sample_b/meshes_v2/preprocessed_cells"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

img = io.imread(img_path)
img = np.einsum('kij->ijk', img)
image = process_labels(img, dilation_iterations=2, erosion_iterations=4, output_directory=save_dir, overwrite = False)

Extending labels: 100%|██████████| 134/134 [13:00<00:00,  5.83s/it]


### Create Meshes

In [6]:
aaa = np.array([[0,0,0,0], [0,1,1,2], [2,1,3,0]])
np.array(aaa == 1).astype(int)

np.isin(aaa, [1,2])

array([[False, False, False, False],
       [False,  True,  True,  True],
       [ True,  True, False, False]])

In [11]:
def create_combined_mesh(
        cell_labels, 
        labels_path, 
        make_meshes=True, 
        make_combined=True, 
        smoothing_iters=10,
        voxel_size=(0.21, 0.21, 0.39)):
    """
    Function to convert a preprocessed array of cell labels into meshes.

    args:
        cell_labels: an iterable containing the ids of the cells to be meshed, or "all" if all the labels 
        in the array need to be converted.
        labels_path: path to the array.
        make_meshes: a boolean telling if the meshes for the single cells are required.
        make_combined: a boolean telling if the combined mesh of cell_labels is required.
        smoothing_iters: an int representing the number of smoothing iterations to be done when creating the mesh.
        voxel_size: an iterable of len (3,) containing the voxel size in (x, y, z) ordering.

    returns:
        meshes: a list whose elements are the meshes associated to each label.
        combined_mesh: the mesh associated to the combined labels.
        cell_labels: a list containing the indexes of the processed labels. 
    """
    
    # Read the image
    labels = np.load(labels_path)

    # Set list of label ids
    if cell_labels == "all":
        cell_labels = np.unique(labels)

    # Make a combined mask for all the labels in cell_labels
    combined_mask = np.isin(labels, cell_labels).astype(int)
    filtered_labels = labels * combined_mask
    
    # Create meshese from the filtered_labels array using trimesh
    meshes = None
    if make_meshes:
        meshes = convert_cell_labels_to_meshes(filtered_labels, 
                                               voxel_resolution=voxel_size, 
                                               smoothing_iterations=smoothing_iters)
    
    # Get the combined mesh from combined_mask array
    combined_mesh = None
    if make_combined:
        combined_mesh = convert_cell_labels_to_meshes(combined_mask, 
                                                      voxel_resolution=voxel_size, 
                                                      smoothing_iterations=smoothing_iters)
    
    return meshes, combined_mesh, cell_labels

In [13]:
# cells = [530, 163, 110, 146, 594, 109, 138, 115, 157, 533, 94, 145, 155, 164, 200, 129, 178, 201, 241, 522]
image_path = '../meshes_for_simulation/lung_new_sample_b/preprocessed_cells/processed_labels_d2_e4.npy'

combined_mesh, labels_list = create_combined_mesh([1,5,10], image_path, True, False)

### Visualise

In [ ]:
vertices, faces = combined_mesh[0].vertices, combined_mesh[0].faces
viewer = napari.Viewer()
viewer.add_surface((vertices, faces), name=f'large_mesh')

In [ ]:
for i, k in enumerate(cells):
    vertices, faces = labels_list[i][0].vertices, labels_list[i][0].faces
    viewer.add_surface((vertices, faces), name=f'mesh {k}')

### Export

In [ ]:
for i, k in enumerate(cells):
    labels_list[i][0].export(f'/Users/antanas/BC_Project/Simulation_Experiment_3/cell_{k-1}.stl')
combined_mesh[0].export(f'/Users/antanas/BC_Project/Simulation_Experiment_3/large_mesh.stl')

### Reading in to visualise

In [ ]:
path = '/Users/antanas/BC_Project/Simulation_Experiment_2'
viewer_1 = napari.Viewer()

for file in os.listdir('/Users/antanas/BC_Project/Simulation_Experiment_2'):
    mesh = tm.load(os.path.join('/Users/antanas/BC_Project/Simulation_Experiment_2', file))
    vertices, faces = mesh.vertices, mesh.faces
    viewer_1.add_surface((vertices, faces), name=os.path.basename(file))

In [ ]:
import numpy as np
import trimesh as tm
from skimage import io
from napari_process_points_and_surfaces import label_to_surface

path = '/Users/antanas/BC_Project/Control_Segmentation/276_mesh_sim.tif'
labels = io.imread(path)
mask = labels * (labels == 276)


mesh_lst = convert_cell_labels_to_meshes(mask, voxel_resolution=np.array([0.21, 0.21, 0.39]), smoothing_iterations=10)

# # Create a boolean array where True corresponds to the label 530
# mask = labels * (labels == 530) == 530

# # Create a mesh from the mask using trimesh
# mesh = label_to_surface(mask)

# print(mesh)



In [ ]:
mesh_lst[0].export('276_mesh.stl')

In [ ]:
import napari

In [ ]:
viewer = napari.Viewer()
viewer

In [ ]:
cells = [530, 163, 110, 146, 594, 109, 138, 115, 157, 533, 94, 145, 155, 164, 200, 129, 179, 201, 241, 522]

In [ ]:
import trimesh as tm
mesh_list = []

for i in cells:
    mesh_list.append(tm.load_mesh(f'/Users/antanas/BC_Project/Control_Segmentation_final/BC_control_s_5_e_2_d_5/cell_meshes/cell_{i-1}.stl'))

In [ ]:
viewer = napari.Viewer()
for i, k in enumerate([276, 394, 301, 595, 404, 311, 355, 404, 437, 394, 524, 357, 393, 374, 501, 237, 509, 356]):
    vertices, faces = mesh_list[i].vertices, mesh_list[i].faces
    viewer.add_surface((vertices, faces), name=f'mesh {k}')

In [ ]:
m178 = tm.load_mesh(f'/Users/antanas/BC_Project/Control_Segmentation_final/BC_control_s_5_e_2_d_5/cell_meshes/cell_{177}.stl')
vertices, faces = m178.vertices, m178.faces
viewer.add_surface((vertices, faces), name='mesh 178')

In [ ]:
vertices, faces = mesh_lst[0].vertices, mesh_lst[0].faces
viewer.add_surface((vertices, faces), name='Large Mesh')

In [ ]:
for i, k in enumerate(cells):
    mesh_list[i].export(f'/Users/antanas/BC_Project/Simulation_Experiment/cell_{k-1}.stl')
mesh_lst[0].export(f'/Users/antanas/BC_Project/Simulation_Experiment/large_mesh.stl')

In [ ]:
import os
import napari 

path = '/Users/antanas/BC_Project/Simulation_Experiment_2'
viewer_1 = napari.Viewer()

for file in os.listdir('/Users/antanas/BC_Project/Simulation_Experiment_2'):
    mesh = tm.load(os.path.join('/Users/antanas/BC_Project/Simulation_Experiment_2', file))
    vertices, faces = mesh.vertices, mesh.faces
    viewer_1.add_surface((vertices, faces), name=os.path.basename(file))